In [1]:
DATASET_FOLDER = './datasets'
INPUT_NAME = 'blur'
OUTPUT_NAME = 'sharp'


In [9]:
import glob
import os
import pandas as pd
import numpy as np

In [28]:
def getLastComp(path):
    return path.split('/')[-1]

def getLastDir(path):
    return path.split('/')[-2]

In [3]:
datasets = glob.glob(os.path.join(DATASET_FOLDER, "*/"))

In [4]:
datasets = {getLastDir(x):x for x in datasets}

In [5]:
datasets

{'GOPRO': './datasets/GOPRO/'}

In [34]:
data_as_array = [] # Rellenar un array con los datos para crear un dataframe
for dataset_name, dataset_path in datasets.items():
    for type_path in glob.glob(os.path.join(dataset_path, '*/')):
        for secuence_path in glob.glob(os.path.join(type_path, '*/')):
            for class_path in glob.glob(os.path.join(secuence_path, '*/')):
                for frame_path in sorted(glob.glob(os.path.join(class_path, '*.png'))):
                    data_as_array.append([dataset_name, getLastDir(type_path), getLastDir(secuence_path),
                                          getLastDir(class_path), int(getLastComp(frame_path).split('.')[0]), frame_path])

In [35]:
dataframe = pd.DataFrame(data_as_array ,columns=['dataset', 'type_name', 'secuence','class_name', 'frame_id', 'frame_path'])

In [36]:
dataframe.head()

,dataset,type_name,secuence,class_name,frame_id,frame_path
0,GOPRO,test,GOPR0410_11_00,blur_gamma,101,./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...
1,GOPRO,test,GOPR0410_11_00,blur_gamma,102,./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...
2,GOPRO,test,GOPR0410_11_00,blur_gamma,103,./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...
3,GOPRO,test,GOPR0410_11_00,blur_gamma,104,./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...
4,GOPRO,test,GOPR0410_11_00,blur_gamma,105,./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...


In [40]:
df_train = dataframe[dataframe["type_name"] == "train"]
df_test = dataframe[dataframe["type_name"] == "test"]

In [41]:
len(df_train)

6309

In [43]:
len(df_test)

3333

In [46]:
df_test.loc[1]

dataset                                                   GOPRO
type_name                                                  test
secuence                                         GOPR0410_11_00
class_name                                           blur_gamma
frame_id                                                    102
frame_path    ./datasets/GOPRO/test/GOPR0410_11_00/blur_gamm...
Name: 1, dtype: object